# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# from multiplier import MultiplierProjection
# from entity import Trait

# Settings

In [3]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [4]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [5]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [6]:
gold_standard.shape

(998, 3)

In [7]:
gold_standard.head()

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [8]:
gold_standard["true_class"].value_counts()

1    755
0    243
Name: true_class, dtype: int64

In [40]:
gold_standard["true_class"].value_counts(normalize=True)

1    0.756513
0    0.243487
Name: true_class, dtype: float64

In [9]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load PhenomeXcan data

In [10]:
# input_file = Path(
#     conf.PHENOMEXCAN["SPREDIXCAN_MASHR_ZSCORES_FOLDER"],
#     "most_signif",
#     "spredixcan-most_signif.pkl"
# ).resolve()

# display(input_file)

In [11]:
# phenomexcan_data = pd.read_pickle(input_file)

In [12]:
# phenomexcan_data.shape

In [13]:
# phenomexcan_data = phenomexcan_data.dropna(how='any')

In [14]:
# phenomexcan_data.shape

In [15]:
# phenomexcan_data.head()

In [16]:
# assert phenomexcan_data.index.is_unique

# Load drug-disease predictions

In [17]:
from collections import defaultdict

In [18]:
current_prediction_files = list(OUTPUT_PREDICTIONS_DIR.glob("*.h5"))
display(len(current_prediction_files))

102

In [21]:
predictions = []

for f in current_prediction_files:
    #     print(f.name)

    prediction_data = pd.read_hdf(f, key="prediction")
    prediction_data = pd.merge(
        prediction_data, gold_standard, on=["trait", "drug"], how="inner"
    )

    metadata = pd.read_hdf(f, key="metadata")

    #     new_predictions[f"{metadata.method}"][metadata.data] = prediction_data
    prediction_data["trait"] = prediction_data["trait"].astype("category")
    prediction_data["drug"] = prediction_data["drug"].astype("category")
    prediction_data = prediction_data.assign(method=metadata.method)
    prediction_data = prediction_data.assign(data=metadata.data)

    predictions.append(prediction_data)

#     print(f"  shape: {prediction_data.shape}")

In [22]:
predictions = pd.concat(predictions, ignore_index=True)

In [23]:
predictions.shape

(65892, 6)

In [24]:
predictions.head()

,trait,drug,score,true_class,method,data
0,DOID:0050741,DB00215,78.706914,1,Gene-based,smultixcan-mashr-zscores-data
1,DOID:0050741,DB00704,195.548246,1,Gene-based,smultixcan-mashr-zscores-data
2,DOID:0050741,DB00822,-722.484604,1,Gene-based,smultixcan-mashr-zscores-data
3,DOID:10283,DB00014,430.604639,1,Gene-based,smultixcan-mashr-zscores-data
4,DOID:10283,DB00175,972.001058,0,Gene-based,smultixcan-mashr-zscores-data


# Average predictions

In [161]:
def _reduce(x):
    return pd.Series(
        {
            "score": x["score"].max(),
            "true_class": x["true_class"].unique()[0]
            if x["true_class"].unique().shape[0] == 1
            else None,
            "data": x["method"].iloc[0],
        }
    )

In [162]:
predictions_avg = (
    predictions.groupby(["trait", "drug", "method"])
    .apply(_reduce)
    .dropna()
    .sort_index()
    .reset_index()
)

In [163]:
predictions_avg.shape

(1292, 6)

In [164]:
predictions_avg.head()

,trait,drug,method,score,true_class,data
0,DOID:0050741,DB00215,Gene-based,81.043547,1.0,Gene-based
1,DOID:0050741,DB00215,Module-based,0.020711,1.0,Module-based
2,DOID:0050741,DB00704,Gene-based,195.548246,1.0,Gene-based
3,DOID:0050741,DB00704,Module-based,0.034249,1.0,Module-based
4,DOID:0050741,DB00822,Gene-based,80.155587,1.0,Gene-based


# ROC

In [165]:
from sklearn.metrics import roc_auc_score

In [166]:
predictions.groupby(["method", "data"]).apply(
    lambda x: roc_auc_score(x["true_class"], x["score"])
).groupby("method").describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Gene-based,51.0,0.532475,0.028585,0.439941,0.515018,0.536575,0.553617,0.583640
Module-based,51.0,0.533373,0.020747,0.488657,0.521123,0.534607,0.544649,0.577376


In [167]:
predictions_avg.groupby(["method", "data"]).apply(
    lambda x: roc_auc_score(x["true_class"], x["score"])
).groupby("method").describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Gene-based,1.0,0.545523,NaN,0.545523,0.545523,0.545523,0.545523,0.545523
Module-based,1.0,0.603049,NaN,0.603049,0.603049,0.603049,0.603049,0.603049


# PR

In [168]:
from sklearn.metrics import average_precision_score

In [169]:
predictions.groupby(["method", "data"]).apply(
    lambda x: average_precision_score(x["true_class"], x["score"])
).groupby("method").describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Gene-based,51.0,0.814741,0.017095,0.752779,0.805872,0.815171,0.827976,0.845651
Module-based,51.0,0.820030,0.010735,0.793746,0.812190,0.821086,0.827914,0.840807


In [170]:
predictions_avg.groupby(["method", "data"]).apply(
    lambda x: average_precision_score(x["true_class"], x["score"])
).groupby("method").describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Gene-based,1.0,0.804276,NaN,0.804276,0.804276,0.804276,0.804276,0.804276
Module-based,1.0,0.848407,NaN,0.848407,0.848407,0.848407,0.848407,0.848407
